In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
cd ..

/home/wbaik/Code/data_science/playground_pytorch


In [6]:
from postgres_utils import PostgresPandas

In [8]:
ppd = PostgresPandas()

In [ ]:
ppd

In [3]:
fundamental_data = pd.read_csv('10K_data.csv')
price_data = pd.read_csv('./fundamental_data/price_data')

FileNotFoundError: File b'10K_data.csv' does not exist

In [7]:
# Some price data are removed
to_drop_for_insufficient_data = [ticker for ticker in price_data if len(price_data[ticker][price_data[ticker].isnull()]) > 250]

[price_data.drop(ticker, axis=1, inplace=True) for ticker in to_drop_for_insufficient_data]

[None, None, None, None, None, None]

In [10]:
delta_price = price_data.iloc[:,3:].pct_change()
delta_price.dropna(inplace=True)

def sharp_ratio(rets, day_counts=250):
    return rets.mean() / rets.std() * np.sqrt(day_counts)

sharp_rolled = \
    delta_price.rolling(250).apply(sharp_ratio).dropna()
mean = delta_price.rolling(250).mean().dropna() * np.sqrt(250)
vol = delta_price.rolling(250).std().dropna() * np.sqrt(250)

In [12]:
sharp_rolled.shape

(736, 384)

In [46]:
sharp_rolled.to_csv('./fundamental_data/sharp_ratio_rolled')

In [25]:
price_data.head()

,date,Unnamed: 0,date.1,mmm,abt,abbv,acn,atvi,ayi,adbe,...,wltw,wyn,wynn,xel,xrx,xlnx,xyl,yum,zbh,zts
0,2013-01-02,0,2013-01-02,84.326,28.790,NaN,61.197,10.443,67.883,38.34,...,81.347,49.667,102.54,23.068,25.371,32.865,25.544,43.725,64.960,NaN
1,2013-01-03,1,2013-01-03,84.229,29.885,NaN,60.974,10.505,68.039,37.75,...,82.178,49.914,103.67,23.002,25.371,32.802,25.460,43.996,65.890,NaN
2,2013-01-04,2,2013-01-04,84.851,29.706,28.810,61.311,10.657,67.932,38.13,...,82.155,50.571,104.91,23.086,25.089,32.346,25.425,44.480,66.224,NaN
3,2013-01-07,3,2013-01-07,84.959,29.947,28.869,61.047,10.667,67.688,37.94,...,81.912,50.443,105.25,22.841,25.512,32.209,25.090,44.199,66.389,NaN
4,2013-01-08,4,2013-01-08,84.969,29.955,28.242,61.400,10.485,64.241,38.14,...,81.513,50.480,105.17,22.885,25.476,31.554,24.784,42.344,66.485,NaN


In [29]:
indecies = list(delta_price.index)

dates = [price_data['date'][index] for index in indecies]

In [30]:
len(dates)

985

In [14]:
# Recording rolling_correlations : 20 days correlation
rolling_correlations = [np.corrcoef(delta_price.iloc[index-20: index+1].values.T) for index in range(20, delta_price.shape[0])]

In [16]:
len(rolling_correlations)

965

In [39]:
correlation_dfs = {}
# Every 50 correlation
for index in range(0,len(rolling_correlations), 50):
    correlation_dfs[index] = rolling_correlations[index]

In [40]:
keys = list(correlation_dfs.keys())
print(keys)

[0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950]


In [41]:
for key in keys:
    correlation_dfs[dates[key]] = correlation_dfs.pop(key)

In [42]:
correlation_dfs.keys()

dict_keys(['2013-12-11', '2014-02-25', '2014-05-07', '2014-07-18', '2014-09-29', '2014-12-09', '2015-02-23', '2015-05-05', '2015-07-20', '2015-09-29', '2015-12-09', '2016-02-23', '2016-05-04', '2016-07-15', '2016-09-26', '2016-12-06', '2017-02-17', '2017-05-03', '2017-07-14', '2017-09-25'])

In [43]:
correlation_dfs['2013-12-11']

array([[1.        , 0.50756676, 0.36853303, ..., 0.11030911, 0.33873223,
        0.29015084],
       [0.50756676, 1.        , 0.09738407, ..., 0.28047818, 0.62303843,
        0.32267988],
       [0.36853303, 0.09738407, 1.        , ..., 0.15235781, 0.02488475,
        0.19790093],
       ...,
       [0.11030911, 0.28047818, 0.15235781, ..., 1.        , 0.43980742,
        0.32786016],
       [0.33873223, 0.62303843, 0.02488475, ..., 0.43980742, 1.        ,
        0.16202803],
       [0.29015084, 0.32267988, 0.19790093, ..., 0.32786016, 0.16202803,
        1.        ]])

In [45]:
for key, value in correlation_dfs.items():
    pd.DataFrame(value).to_csv(
        './fundamental_data/{}_correlation_matrix'.format(key))